In [2]:
using System.IO;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Linq.Expressions;

var LineSeparator = "\r\n";
var Day = "07";
Elfts ElfDrive;

var Debug = (object s) => 
{
    // Console.WriteLine($"Survived Until here:{s}");
};

public static string[] UltraSplit(this string s, string separator){
    return s.Split(separator, StringSplitOptions.RemoveEmptyEntries);
}

var inputFunction = (string day, bool isTest) => {
    var file = File.ReadAllText($"InputFiles\\D{day}{(isTest ? "_test" : "")}.txt");
    LineSeparator = file.Contains("\r\n") ? "\r\n" : "\n";
    return file;
};

class Elfts 
{
    public ElfDir Root = new (null, "");
}

class ElfDir
{
    public ElfDir(ElfDir parentDirectory, string name){
        this.ParentDirectory = parentDirectory;
        this.Name = name;
    }
    public ElfDir ParentDirectory;
    public string Name;
    public List<ElfDir> Directories = new ();
    public List<ElfFile> Files = new ();
    public IEnumerable<ElfDir> EnumerateDirectories(bool includeSubdirectories = false)
    {
        foreach(var dir in Directories)
        {
            if(includeSubdirectories)
            {
                var subDirs = dir.EnumerateDirectories(includeSubdirectories);
                if(subDirs != null)
                {
                    foreach(var subDir in subDirs){
                        yield return subDir;
                    }
                }
            }
            yield return dir;
        }
    }
    public Tuple<int, bool, ElfDir> GetDirectoryInfo(bool includeSubdirectories = false, int maxSize = 0)
    {
        var size = Files.Sum(f => f.Length) + (includeSubdirectories ? Directories.Sum(f => f.GetDirectoryInfo(includeSubdirectories, maxSize).Item1) : 0);
        return new Tuple<int, bool, ElfDir>(size, size <= maxSize, this);
    }
}

class ElfFile
{
    public ElfFile(string length, string name){
        this.Length = Convert.ToInt32(length.Trim());
        this.Name = name.Trim();
    }
    public int Length;
    public string Name;
}

void LoadFilesystem(string terminalOutput)
{   
    var currentDirectory = ElfDrive.Root;
    
    foreach(var c in terminalOutput.UltraSplit("$ "))
    {
        Debug(c);
        if(c.StartsWith("cd"))
        {
            var whereTo = c.UltraSplit(" ")[1].Trim();
            if(whereTo == "/")
            {
                currentDirectory = ElfDrive.Root;
            }
            else if(whereTo == "..")
            {
                currentDirectory = currentDirectory.ParentDirectory == null ? ElfDrive.Root : currentDirectory.ParentDirectory;
            }
            else
            {
                Debug(currentDirectory.Directories.Count);
                currentDirectory = currentDirectory.Directories.First(d => d.Name == whereTo);
            }
        }
        else 
        {
            var dirContents = c.Replace("ls"+LineSeparator, "").UltraSplit(LineSeparator);
            foreach(var parsedLine in dirContents.Select(l => l.UltraSplit(" ")))
            {
                if(parsedLine[0].StartsWith("dir"))
                {
                    var dir = new ElfDir(currentDirectory, parsedLine[1].Trim());
                    currentDirectory.Directories.Add(dir);
                }
                else 
                {
                    currentDirectory.Files.Add(new ElfFile(parsedLine[0], parsedLine[1]));
                }
            }
        }
    }
}

var puzzleFunction = (bool isTest, Func<string, bool, string> inputFunction, int maxSize) => {
    var input = inputFunction(Day, isTest);
    ElfDrive = new Elfts();
    LoadFilesystem(input);
    var rootInfo = ElfDrive.Root.GetDirectoryInfo(true, maxSize);
    var dirsUnderMaxSize = ElfDrive.Root.EnumerateDirectories(true).Select(d => d.GetDirectoryInfo(true, maxSize)).Where(d => d.Item2);
    var sum = rootInfo.Item2 ? rootInfo.Item1 : 0 + dirsUnderMaxSize.Sum(d => d.Item1);
    Console.WriteLine((isTest ? "Test" : "Puzzle") + $" Result: {sum}");
};

Console.WriteLine("Problem 01:");
puzzleFunction(true, inputFunction, 100000);
puzzleFunction(false, inputFunction, 100000);


Console.WriteLine("");
Console.WriteLine("Problem 02:");

puzzleFunction = (bool isTest, Func<string, bool, string> inputFunction, int maxSize) => {
    var input = inputFunction(Day, isTest);
    ElfDrive = new Elfts();
    LoadFilesystem(input);
    var rootInfo = ElfDrive.Root.GetDirectoryInfo(true, maxSize);
    var requiredMinSize = Math.Abs(70000000 - 30000000 - rootInfo.Item1);
    var dirSize = ElfDrive.Root.EnumerateDirectories(true).Select(d => d.GetDirectoryInfo(true)).Where(t => t.Item1 >= requiredMinSize).Min(t => t.Item1);
    
    Console.WriteLine((isTest ? "Test" : "Puzzle") + $" Result: {dirSize}");
};

puzzleFunction(true, inputFunction, 100000);
puzzleFunction(false, inputFunction, 100000);




Problem 01:
Test Result: 95437
Puzzle Result: 1141028

Problem 02:
Test Result: 24933642
Puzzle Result: 8278005
